# Linear Regression - some Theory
* [other compariative exmaples](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/)
For linear regression the closed solution is known via the so called normal equation
Todo:
1. Specify model
2. Speficy loss function
2. Write down log Likelihood function
3. Minimize + get Normal equation

### Approach 1: Non-probabilisitc using loss function
The easies approach to linear regression is to use the squared loss function, 
$$l = \sum_{i=1}^N (y_i - \hat y_i)^2,$$
where $\hat y_i$ is the predicted value and $y_i$ is the true value of sample $i$. Together with a linear, non probabilistic model for the. For feature vector $x_i$ (where the first component is per convention the constant one -aka known as intercept) the prediction is given by the linear model. In scalar product notation:
$$y_i =  x_i^T \theta,$$
with coefficient vector $\theta$. Rewriting this in Matrix notation to account for all sampels we get for the loss together with the linear model:
$$l = (Y - X\theta)^T (Y - X\theta)$$
Now we would like to find $\theta^*$ that _minimizes_ the loss. 
Setting the derivative w.r.t. $\theta$ of the loss function zero  gives the normal equation 
$$X^T(Y - X\theta) = 0.$$
If $X^T X$ is non-singular (this is the case when there are more traning examples than features because then $X^T X$ is positive definite) allows for finding the unique solutions of the normal equation, 

$$\theta ^* =  \left( X^T X  \right)^{-1} X^T Y$$

Note that this approach is non-probabilistic and thus, does not explicitly account for uncertainty  (as a probability distributions) in the data and coefficients.
### Approach 2: Probabilistic  +  generative approach using max likelihood
This approach introduces a probability distribution but does not explicitly consider a loss function. The  response is modelled  via a Normal distribution ("Gauss error") assuming constant standard deviation
$$y_i = x_i^T \theta + \epsilon := \mathcal N (x_i^T \theta, \sigma^2)$$
In other words, the conditional distribtion  $p(y \mid x, \theta, \sigma^2)$ is given by a Normal distribtion.
The likelihood function is just the pdf of __all__ datapoints assuming i.i.d (this assumption in fact leads to the factorization), 
$$\mathcal L = \Pi_{i=1}^N p(y_i \mid x, \theta, \sigma).$$
As we are aiming to optimize $\theta$ in a way, a striclty monotonic transformation is applied on the likelihood function. It leaves the optimum invariant. The standard procedure is thus to consider the the logarithm ot likelihood function:
$$\mathcal L_l = \sum_{i=1}^N \log p(y_i \mid x, \theta, \sigma).$$
Evaluating this expression for the Normal distribution gives
$$\mathcal L_l =  - \frac{1}{2\sigma^2}\sum_{i=1}^N (y_i - x_i^T \theta)^2  - \frac{N}{2}log(2\pi\sigma^2) $$
Now, we would like to _maximise_ the likelihood and thus the log likelihood with respect to $\theta$. This is equivalent to _minimizing_ the negative of it. Throwing away terms that, don't depend on $\theta$ gives the function for which we would like to find the minimizer. That is we want to solve this expression, 
$$\text{argmin}_\theta\left( Y - X \theta \right )^T \left( Y - X \theta \right ),$$
where we have rewritten the sum of squares over all training data again in Matrix notation. But this is exactly the same problem as in approach one and this gives the same solution (under the same circumstances), 

$$\theta ^* =  \left( X^T X  \right)^{-1} X^T Y$$

**Remarks**
* Note that this procedure only puts a probablilty distribution on the response $y$ while treating the remaining ingredients as variables (via the conditional pdf Ansatz). 
* This already implies that in this context the solutions $\theta^*$ just tell how the pdf is parametrized (not even completely as we did not consider the optimum value of $\sigma$).
* In either case this approach does not really tell us how to predict a specific value $y$ for a given $x$. It just tells us the corresponding distribution of $y$. The fundamental reason is that we did not make any use of a loss function in this approach. 
* Pragmatically and in practice  of course, the prediction is made by plugging into the linear model as e.g. in the first appraoch
* Note that a constant value for $\sigma$ is called homoscedasticity. This implies that the variance does may not be a function of the features but only the mean within the Normal model for the response.
* Furhtermore note that this approach does not consider uncertainties in the paramters. This would eventually require a Bayesian approach. 

### Approach 3: Probabilistic  +  discriminative approach using loss function and max likelihood

### Apporach 4: A Bayesian approach

Approaches: 
- Via max likelihood https://www.quantstart.com/articles/Maximum-Likelihood-Estimation-for-Linear-Regression
- Minimize quadratic error directly

# Get the data

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
Supervised = namedtuple("supervised", ["features", "target", "feature_names"])

def split_test_train(data):
    X_train, X_test, Y_train, Y_test = train_test_split(data.features, data.target, test_size = 0.2, random_state=5)
    train = Supervised(X_train, Y_train.reshape(-1, 1), feature_names=data.feature_names)
    test = Supervised(X_test, Y_test.reshape(-1, 1), feature_names=data.feature_names)
    return train, test 


def normalize(train: Supervised, test: Supervised):
    mu = train.features.mean(axis=0)
    std = train.features.std(axis=0)
    train_scaled = Supervised(features=(train.features - mu) / std, 
                              target=train.target, 
                              feature_names=train.feature_names) 
    test_scaled = Supervised(features=(test.features - mu) / std, 
                             target=test.target, 
                             feature_names=test.feature_names)
    return train_scaled, test_scaled


housing = fetch_california_housing()
train_data, test_data = normalize(*split_test_train(Supervised(housing.data, housing.target, housing.feature_names)))

# Linear Regression via the Normal equation - with Tensorflow

In [2]:
def add_intercept(features):
    """Add intercept to features
    Todo: as an exercise use tensorflow"""
    m, n = features.shape
    return np.c_[np.ones((m, 1)), features]

    
def train(data):
    X = add_intercept(data.features)
    Y = data.target
    XT = tf.transpose(X)
    return tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), Y)


def predict(data, theta):
    return tf.matmul( add_intercept(data.features),  theta)

theta_analytic = train(train_data)
y_pred_analytic  = predict(test_data, theta_analytic)

# Linear Regression as a Neural Network with Tensorflow / Keras 
* Idea: use keras wit 1 layer, identitiy as activation function
* Assume that we could not find the minimizer analytically. 
* Thus we wish to find the minimzer computationally.

> **Note** the accuracy with an explicit optimizier critically hinges on the convergence behaviour of the optimizer.  In this case wie use the Adam optimizer with learning rate = 0.01. It's instructive to break the model by employing a non-convergent optimizer (e.g. RSMprops with learnin rate 0.01 for this particular case).

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


def plot_keras_training(history):
    plt.figure(figsize=(15,4))
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
    plotter.plot({'Basic': history}, metric = "mse")
    #plt.ylim([0, 3])
    #plt.xlim([0,30])
    plt.ylabel('MSE')
    plt.show()

def build_model():
    model = keras.Sequential([layers.Dense(1, activation='linear', input_shape=[8])])
    optimizer = tf.keras.optimizers.Adam(0.001)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mse'])
    return model

model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [22]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

EPOCHS = 10000

history = model.fit(
  train_data.features, train_data.target,
  batch_size = train_data.features.shape[0],
  epochs=EPOCHS, 
  validation_split = 0., 
  verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

theta_keras = tf.concat((model.weights[1], tf.reshape(model.weights[0], [8])), axis=0)
y_pred_keras = model.predict(test_data.features)


Epoch: 0, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 100, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 200, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 300, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 400, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 500, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 600, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 700, lo

....................................................................................................
Epoch: 5900, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 6000, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 6100, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 6200, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 6300, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 6400, loss:0.5214,  mse:0.5214,  
....................................................................................................
Epoch: 6500, loss:0.5214,  mse:0.5214,  
......

In [23]:
# Todo: plot even if there is no validation set
# plot_keras_training(history)

### Remarks
* In order to get the "best" agreement with other implementations it is crucial to have implemented the _same approch_:
1. Pass all data to optimization step 
    `batch_size = train_data.features.shape[0]` and
    `validation_split = 0.` 
2. The required number of epochs is then getting much larger. 
3. In other words: we are not doing batch regression.


# Linear Regression using the estimator API
* within the estimator API the number of training steps is controlled via the the `dataset`
* more precisely via the `repeat` keyword

In [24]:
from tensorflow.estimator import LinearRegressor
from tensorflow import feature_column

In [27]:
def input_fn(data: Supervised, train=False):
    """Todo: make callable class for more intuitive usage lateron"""
    raw_xs = pd.DataFrame(data.features, columns = data.feature_names)
    xs = {key:np.array(value) for key,value in dict(raw_xs).items()} 
    ys = data.target
    ds = tf.data.Dataset.from_tensor_slices((xs, ys)).batch(raw_xs.shape[0])
    if train:
        return ds.repeat(1000)
    return ds
    

In [28]:
feature_columns = [feature_column.numeric_column(f) for f in housing.feature_names]
linear_regressor = LinearRegressor(feature_columns=feature_columns)
linear_regressor.train(lambda: input_fn(train_data, train=True))
y_pred_estimator = np.array([r["predictions"] for r in linear_regressor.predict(lambda: input_fn(test_data, train=False))])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp8su75vi7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. 

In [29]:
for name in linear_regressor.get_variable_names():
    print(name, linear_regressor.get_variable_value(name))

global_step 1000
linear/linear_model/AveBedrms/weights [[0.32431135]]
linear/linear_model/AveOccup/weights [[-0.04502133]]
linear/linear_model/AveRooms/weights [[-0.2811665]]
linear/linear_model/HouseAge/weights [[0.11540022]]
linear/linear_model/Latitude/weights [[-0.89616174]]
linear/linear_model/Longitude/weights [[-0.86808366]]
linear/linear_model/MedInc/weights [[0.82789296]]
linear/linear_model/Population/weights [[-0.00344467]]
linear/linear_model/bias_weights [2.0638926]
training/Ftrl/decay 0.0
training/Ftrl/l1_regularization_strength 0.0
training/Ftrl/l2_regularization_strength 0.0
training/Ftrl/learning_rate 0.2
training/Ftrl/learning_rate_power -0.5
training/Ftrl/linear/linear_model/AveBedrms/weights/accumulator [[0.16681933]]
training/Ftrl/linear/linear_model/AveBedrms/weights/linear [[-0.6623008]]
training/Ftrl/linear/linear_model/AveOccup/weights/accumulator [[0.10407151]]
training/Ftrl/linear/linear_model/AveOccup/weights/linear [[0.07261966]]
training/Ftrl/linear/linear

# Sklearn 

In [30]:
# For comparison perform linear regression with scikit learn
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression(fit_intercept=True, normalize=False)
lin_model.fit(train_data.features, train_data.target)
theta_sklearn = tf.concat((lin_model.intercept_, lin_model.coef_.flatten()), axis=0)
y_pred_sklearn = lin_model.predict(test_data.features)

# Comparision of results

In [31]:
def mse(y_true, y_pred):
    return tf.reduce_sum((y_true - y_pred)**2) / y_true.shape[0]

In [32]:
mse(test_data.target,  y_pred_sklearn) 

<tf.Tensor: shape=(), dtype=float64, numpy=0.536342730250191>

In [33]:
mse(test_data.target,  y_pred_analytic) 

<tf.Tensor: shape=(), dtype=float64, numpy=0.5363427302501911>

In [34]:
mse(test_data.target,  y_pred_keras) 

<tf.Tensor: shape=(), dtype=float64, numpy=0.5363423472592311>

In [35]:
mse(test_data.target,  y_pred_estimator) 

<tf.Tensor: shape=(), dtype=float64, numpy=0.5363428266543355>

In [ ]:
y_pred_keras.shape

In [ ]:
y_pred_estimator.shape

In [ ]:
theta_sklearn